# Tensor fundamentals with Tensorflow

More specifically, we're going to cover:
* Introduction to tensors
* Geeting information from tensors 
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to spped up your regular Python functions)
* Using GPUs with Tensorflow (or TPUs)
* Exercises to try for yourself

## Introduction to Tensors 

In [14]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.4.1


In [15]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [16]:
# Check number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [17]:
# Create a vector 
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [18]:
# Check dimension of our vector 
vector.ndim

1

In [19]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant ([[10, 7],
                       [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [20]:
matrix.ndim

2

In [21]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [22]:
# What's the number dimensions of another matrix?
another_matrix.ndim

2

In [23]:
# Let's create a Tensor 
tensor = tf.constant ([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                      [10, 11, 12]],
                      [[13, 14, 15],
                      [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [24]:
tensor.ndim

3

What  we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is scalar, a 1-dimensional tensor is a vector...)

### Creating tensors with 'tf.Variable'


In [25]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor 

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [26]:
# Let's try to change one of the elements in our changeable tensor 
changeable_tensor[0] = 7 
changeable_tensor 

TypeError: 'ResourceVariable' object does not support item assignment

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor 

In [ ]:
# Let's try to change our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

🔑 **Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as Tensorflow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Ranndom tensors are tensors of some abitrary size wiich contain random numbers

In [ ]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducibility ()
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1, random_2, random_1 == random_2,

## Shuffle the order of elements in a tensor 

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant ([[10, 7],
                            [3, 4],
                            [2, 5]])
#Shuffle our non_shuffled tensor 
tf.random.shuffle(not_shuffled)

In [ ]:
#Shuffle our non_shuffled tensor 
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

In [ ]:
not_shuffled

🛠️ **Exercise:** Read through Tensorflow documentation on random seed generation:
https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writign 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in hte same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [ ]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) #operation level random seed 

### Other ways to make tensor

In [ ]:
# Create a tensor of all ones
tf.ones([10,7])

In [ ]:
# Create a tensor of all zeros
tf.zeros(shape=(3,4))

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and Tensorflow tensor is that tensors can be runed on GPU computing

In [28]:
# You cand also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a nnmpy array between 1 and 25
numpy_A
# x = tf.contant(some_matrix) # capital for matrix or tensor 
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [29]:
2 * 3 * 4

24

In [30]:
3 * 8

24

In [31]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A, shape=(3, 8))
C = tf.constant(numpy_A)
A, B, C

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]])>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>)

### Getting informations from tensors

When dealing with tensors you probably want to be aware of the following attributes:

* **Shape** / The length (number of elements) of each of the dimensions of a tensor / `tensor.shape`
* **Rank** / the number of the tensor dimension ( scalar, vector, matrix, n...) / `tensor.ndim`
* **Axis or dimension** / A particular dimension of a tensor / `tensor[0]`, `tensor[:, 1]`...
* **Size** / Total number of items in the tensor / `tf.size(tensor`

In [32]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=(2,3,4,5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [33]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [34]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [35]:
2 * 3 * 4 * 5 

120

In [36]:
# Get various attributes of our tensor]
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor)) 
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy()) # add `.numpy` for single output

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


### Indexing tensor 

Tensors can be indexed just like Python lists

In [37]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [38]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [39]:
# Get the first element from each dimensions from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [40]:
#Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                           [3, 4]])
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [41]:
print("Datatype of every element:", rank_2_tensor.dtype)
print("Number of dimensions (rank):", rank_2_tensor.ndim)
print("Shape of tensor:", rank_2_tensor.shape)
print("Elements along the 0 axis:", rank_2_tensor.shape[0])
print("Elements along the last axis:", rank_2_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_2_tensor)) 
print("Total number of elements in our tensor:", tf.size(rank_2_tensor).numpy()) # add ".numpy" for single output

Datatype of every element: <dtype: 'int32'>
Number of dimensions (rank): 2
Shape of tensor: (2, 2)
Elements along the 0 axis: 2
Elements along the last axis: 2
Total number of elements in our tensor: tf.Tensor(4, shape=(), dtype=int32)
Total number of elements in our tensor: 4


In [42]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

In [43]:
# Add in extra dimensions to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # "..." add new axis to the end
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [44]:
# Alternative to "tf.newaxis"
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [45]:
# Expand the 0 axis
tf.expand_dims(rank_2_tensor, axis=0) 

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]])>

### Manipulating tensor (tensor operations)

### Basic operations 

`+`, `-`, `*`, `/`

In [73]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7],
                     [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [74]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [75]:
# Multiplication
tensor * 10 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [76]:
# Substraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]])>

In [77]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [78]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

### Matrix multiplication

In machine learning, matrix multiplication is one of the most common tensor operations

There aret wo rules our tensor (or matrices) need to fulfil if we're going to matrix multiplay them:

1. The inner dimension must match
2. The resulting matrix has the shape of the outter dimensions

In [80]:
# Matrix multiplication with tensorlfow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [81]:
# Matrix multiplication with Python operator "@"
tensor @ tensor 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [88]:
# Create a tensor of (3, 2)
x = tf.constant([[1, 2],
                [3, 4],
                [5, 6]])
# Create another (2, 3)
y = tf.constant ([[7, 8],
                 [9, 10],
                 [11, 12]])
x, y 

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [89]:
# Reshape the tensor Y
# Same as: `tf.transpose(tensor)`
tf.reshape(y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]])>

In [87]:
# Try to matrix multiply tensors of the same shape 
tf.matmul(x, y)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

### The dot product

Matrix multiplication is also reffered to as the dot product

You can perform matrix multiplication using:

- `tf.matmul()`
- `tf.tensordot()`

In [94]:
print(x)
print(y)

tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32)


In [95]:
tf.matmul(x, tf.transpose(y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

In [98]:
x @ tf.transpose(y)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

### Changing the datatype of a tensor 


In [99]:
# Create a new tensor with the default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [100]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [103]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [106]:
# Change from int32 to float32 
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

### Aggregating tensors